In [ ]:
from chathist import InstructionDataLoader,InstructionDataset, InstructionStyle, Model, Tokenizer
import pandas as pd

In [ ]:
chat_df = pd.read_json("hf://datasets/ogrnz/chat-titles/dataset.jsonl", lines=True)

In [ ]:
chat_df.head()

In [ ]:
chat_df.shape

In [ ]:
chat_df.columns

In [ ]:
style = InstructionStyle.load(style='phi3', input_query="<|user|>\n", response_query="\n<|llm|>\n")

In [ ]:
df = style.convert_train(chat_df, input_col='message', response_col= 'title', output_col='instruct', new_df=True)

In [ ]:
train_len = 9500

In [ ]:
train_df = df.iloc[:train_len, :]
val_df = df.iloc[train_len:, :]

In [ ]:
train_df

In [ ]:
print(train_df['instruct'].values[0])

In [ ]:
val_df

In [ ]:
print(val_df['instruct'].values[0])

In [ ]:
train_dataset = InstructionDataset(train_df['instruct'].tolist())

In [ ]:
val_dataset = InstructionDataset(val_df['instruct'].tolist())

In [ ]:
train_loader = InstructionDataLoader().load(dataset = train_dataset)

In [ ]:
val_loader = InstructionDataLoader().load(dataset = val_dataset)

In [ ]:
data = iter(val_loader)

In [ ]:
first_batch = next(data)

In [ ]:
first_batch[0][0]

In [ ]:
first_batch[1][0]

In [ ]:
tokenizer = Tokenizer()

In [ ]:
print(tokenizer.decode_ids(first_batch[0][0]))

In [ ]:
print(tokenizer.decode_ids(first_batch[1][0]))

In [ ]:
model = Model()

In [ ]:
for name, layer in model._model.named_parameters():
    print(name)
    print(layer.requires_grad)

In [ ]:
train_loss, val_loss = model.train(train_loader = train_loader, val_loader = val_loader)